In [4]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# ---------------------
# 1. Data Preparation
# ---------------------
def parse_data(filename):
    """Parse Qian & Sejnowski format files"""
    with open(filename, 'r') as f:
        content = f.read()
    
    sequences = []
    current_seq = []
    for line in content.split('\n'):
        if line.strip() == '<end>':
            if current_seq:
                sequences.append(current_seq)
                current_seq = []
        elif line and not line.startswith('#') and not line.startswith('<'):
            parts = line.strip().split()
            if len(parts) >= 2:
                aa = parts[0]
                ss = parts[1] if len(parts[1]) > 0 else '_'
                current_seq.append((aa, ss))
    return sequences

# Load data - replace with actual file paths
train_sequences = parse_data('protein-secondary-structure.train.txt')
test_sequences = parse_data('protein-secondary-structure.test.txt')

# ---------------------
# 2. Encoding Utilities
# ---------------------
AMINO_ACIDS = ['A','C','D','E','F','G','H','I','K','L','M',
               'N','P','Q','R','S','T','V','W','Y','X']  # X as spacer
SS_CLASSES = ['h', 'e', '_']  # helix, sheet, coil

def encode_window(window):
    """Qian & Sejnowski's local encoding scheme"""
    encoded = []
    for aa, _ in window:
        vec = [0]*len(AMINO_ACIDS)
        try:
            idx = AMINO_ACIDS.index(aa)
        except ValueError:
            idx = -1  # handle unknown as spacer
        vec[idx] = 1
        encoded.extend(vec)
    return encoded

# ---------------------
# 3. Original 1988 Network
# ---------------------
class QSNet(nn.Module):
    """Exact replica of Qian & Sejnowski (1988) architecture"""
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13*21, 40)  # 13 residues × 21 amino acids
        self.fc2 = nn.Linear(40, 3)      # 3 output classes
        self.sigmoid = nn.Sigmoid()       # Original activation
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

# ---------------------
# 4. Dataset Preparation
# ---------------------
def create_dataset(sequences, window_size=13):
    """Create sliding window dataset with padding"""
    X, y = [], []
    for seq in sequences:
        # Add padding (6 spacers on each side)
        padded = [('X', '_')]*6 + seq + [('X', '_')]*6
        seq_len = len(seq)
        
        for i in range(seq_len):
            # Get 13-residue window centered on current amino acid
            window = padded[i:i+window_size]
            encoded = encode_window(window)
            label = SS_CLASSES.index(window[6][1])  # Center residue label
            X.append(encoded)
            y.append(label)
    
    return np.array(X, dtype=np.float32), np.array(y)

# Create datasets
X_train, y_train = create_dataset(train_sequences)
X_test, y_test = create_dataset(test_sequences)

# Convert to PyTorch datasets
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train, dtype=torch.long))  # Add dtype=torch.long
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test, dtype=torch.long))     # Add dtype=torch.long


# ---------------------
# 5. Training Setup
# ---------------------
def train_model(model, train_loader, test_loader, epochs=50, model_name='Base'):
    """Training function with metrics tracking"""
    train_losses = []
    test_accs = []
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    
    for epoch in range(epochs):
        # Training
        model.train()
        epoch_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.numpy())
                all_labels.extend(labels.numpy())
        
        # Calculate metrics
        train_loss = epoch_loss/len(train_loader)
        acc = accuracy_score(all_labels, all_preds)
        train_losses.append(train_loss)
        test_accs.append(acc)
        
        print(f'Epoch {epoch+1:02d} | Loss: {train_loss:.3f} | Acc: {acc:.3f}')

    # Plot learning curves
    plt.figure()
    plt.plot(train_losses, label='Training Loss')
    plt.plot(test_accs, label='Test Accuracy')
    plt.title(f'{model_name} Learning Curves')
    plt.xlabel('Epoch')
    plt.legend()
    plt.savefig(f'{model_name}_curves.png')
    plt.close()

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=SS_CLASSES, yticklabels=SS_CLASSES)
    plt.title(f'{model_name} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f'{model_name}_cm.png')
    plt.close()
    
    return max(test_accs)

# ---------------------
# 6. Original Model Training
# ---------------------
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Initialize and train original model
original_model = QSNet()
orig_acc = train_model(original_model, train_loader, test_loader, 
                      epochs=50, model_name='Original')

# ---------------------
# 7. Improved Model with Positional Encoding
# ---------------------
class ImprovedQSNet(nn.Module):
    """Enhanced version with positional information"""
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13*21 + 1, 40)  # Original + position
        self.fc2 = nn.Linear(40, 3)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

def create_improved_dataset(sequences):
    """Dataset with positional encoding"""
    X, y = [], []
    for seq in sequences:
        padded = [('X', '_')]*6 + seq + [('X', '_')]*6
        seq_len = len(seq)
        
        for i in range(seq_len):
            # Original encoding
            window = padded[i:i+13]
            encoded = encode_window(window)
            
            # Add positional feature (normalized 0-1)
            position = i / seq_len
            encoded.append(position)
            
            label = SS_CLASSES.index(window[6][1])
            X.append(encoded)
            y.append(label)
    
    return np.array(X, dtype=np.float32), np.array(y)

# Create improved datasets
X_train_imp, y_train_imp = create_improved_dataset(train_sequences)
X_test_imp, y_test_imp = create_improved_dataset(test_sequences)

train_dataset_imp = TensorDataset(torch.tensor(X_train_imp), torch.tensor(y_train_imp, dtype=torch.long))
test_dataset_imp = TensorDataset(torch.tensor(X_test_imp), torch.tensor(y_test_imp, dtype=torch.long))

train_loader_imp = DataLoader(train_dataset_imp, batch_size=BATCH_SIZE, shuffle=True)
test_loader_imp = DataLoader(test_dataset_imp, batch_size=BATCH_SIZE)

# Train improved model
improved_model = ImprovedQSNet()
imp_acc = train_model(improved_model, train_loader_imp, test_loader_imp,
                     epochs=50, model_name='Improved')

# ---------------------
# 8. Results Comparison
# ---------------------
# Generate comparison table
results = [
    {'Model': 'Original (1988)', 'Accuracy': orig_acc},
    {'Model': 'Improved (Positional)', 'Accuracy': imp_acc}
]

print("\nModel Comparison:")
print(f"{'Model':<20} | {'Accuracy':>10}")
print("-"*33)
for res in results:
    print(f"{res['Model']:<20} | {res['Accuracy']:>10.3f}")

# Generate comparison plot
plt.figure()
models = [res['Model'] for res in results]
accs = [res['Accuracy'] for res in results]
plt.bar(models, accs)
plt.ylim(0.5, 0.7)
plt.title('Model Performance Comparison')
plt.ylabel('Accuracy')
plt.savefig('model_comparison.png')
plt.close()

Epoch 01 | Loss: 1.002 | Acc: 0.546
Epoch 02 | Loss: 0.969 | Acc: 0.547
Epoch 03 | Loss: 0.906 | Acc: 0.604
Epoch 04 | Loss: 0.852 | Acc: 0.621
Epoch 05 | Loss: 0.827 | Acc: 0.621
Epoch 06 | Loss: 0.821 | Acc: 0.626
Epoch 07 | Loss: 0.817 | Acc: 0.623
Epoch 08 | Loss: 0.816 | Acc: 0.621
Epoch 09 | Loss: 0.815 | Acc: 0.626
Epoch 10 | Loss: 0.815 | Acc: 0.624
Epoch 11 | Loss: 0.813 | Acc: 0.620
Epoch 12 | Loss: 0.814 | Acc: 0.620
Epoch 13 | Loss: 0.814 | Acc: 0.620
Epoch 14 | Loss: 0.811 | Acc: 0.615
Epoch 15 | Loss: 0.812 | Acc: 0.626
Epoch 16 | Loss: 0.811 | Acc: 0.628
Epoch 17 | Loss: 0.811 | Acc: 0.625
Epoch 18 | Loss: 0.811 | Acc: 0.623
Epoch 19 | Loss: 0.811 | Acc: 0.630
Epoch 20 | Loss: 0.809 | Acc: 0.621
Epoch 21 | Loss: 0.811 | Acc: 0.624
Epoch 22 | Loss: 0.809 | Acc: 0.619
Epoch 23 | Loss: 0.810 | Acc: 0.628
Epoch 24 | Loss: 0.809 | Acc: 0.625
Epoch 25 | Loss: 0.808 | Acc: 0.624
Epoch 26 | Loss: 0.808 | Acc: 0.614
Epoch 27 | Loss: 0.808 | Acc: 0.623
Epoch 28 | Loss: 0.808 | Acc